In [1]:
import graphviz
from math import lcm
from functools import reduce


In [2]:
file1 = open('input.txt', 'r')
lines = file1.readlines()

In [3]:
instructions = {}
for line in lines:
    instruction = line.strip().split(' -> ')
    if instruction[0] == "broadcaster":
        instructions[instruction[0]] = {"type": "broadcaster", "value": instruction[1].split(", ")}
    elif instruction[0][0] == "%":
        instructions[instruction[0][1:]] = {"type": instruction[0][0], "value": instruction[1].split(", "), "state": "off"}#flipflop
    elif instruction[0][0] == "&":
        instructions[instruction[0][1:]] = {"type": instruction[0][0], "value": instruction[1].split(", "), "last": {}}#Conjunction

for key in instructions:
    if instructions[key]["type"] == "&":
        for i in list(dict(filter(lambda x: key in x[1]["value"], instructions.items())).keys()):
            instructions[key]["last"][i] = "low"

def process_button(current_command, source, signal, future_commands):
    running = True
    counters = (0,0)
    while running:
        #print(f"{source} -> {signal} -> {current_command}]")
        change = (0,1) if signal == "low" else (1,0)
        counters = tuple(map(sum, zip(counters, change)))

        if current_command in instructions:
            #print("out")
            if instructions[current_command]["type"] == "broadcaster":
                for i in instructions[current_command]["value"]:
                    future_commands.append((i, current_command, signal))

            elif instructions[current_command]["type"] == "&":#Conjunction
                instructions[current_command]["last"][source] = signal
                if all(list(map(lambda x: x == "high", instructions[current_command]["last"].values()))):
                    for i in instructions[current_command]["value"]:
                        future_commands.append((i, current_command, "low"))
                else:
                    for i in instructions[current_command]["value"]:
                        future_commands.append((i, current_command, "high"))

            elif instructions[current_command]["type"] == "%": #flipflop
                if signal == "low":
                    if instructions[current_command]["state"] == "off":
                        instructions[current_command]["state"] = "on"
                        for i in instructions[current_command]["value"]:
                            future_commands.append((i, current_command, "high"))
                    else:
                        instructions[current_command]["state"] = "off"
                        for i in instructions[current_command]["value"]:
                            future_commands.append((i, current_command, "low"))
        #print(future_commands)
        if len(future_commands) > 0:
            next_command = future_commands.pop(0)
            current_command = next_command[0]
            source = next_command[1]
            signal = next_command[2]
        else:
            running = False

    return counters

counters = (0, 0)
for i in range(1000):
    counters = tuple(map(sum, zip(counters, process_button("broadcaster", "button", "low", []))))
print(f"part1: {counters[0]*counters[1]}")


part1: 929810733


In [4]:
dot = graphviz.Digraph(comment='The Machine')
for key in instructions:
    dot.node(key, key)
for key in instructions:
    for i in instructions[key]["value"]:
        dot.edge(key, i)

dot.render('round-table.gv', view=True)

'round-table.gv.pdf'

In [5]:
def process_button(current_command, source, signal, future_commands):
    running = True
    counters = (0,0)
    while running:
        #print(f"{source} -> {signal} -> {current_command}]")
        change = (0,1) if signal == "low" else (1,0)
        counters = tuple(map(sum, zip(counters, change)))

        if current_command == "lg" and signal == "high":
            return True

        if current_command in instructions:
            #print("out")
            if instructions[current_command]["type"] == "broadcaster":
                for i in instructions[current_command]["value"]:
                    future_commands.append((i, current_command, signal))

            elif instructions[current_command]["type"] == "&":#Conjunction
                instructions[current_command]["last"][source] = signal
                if all(list(map(lambda x: x == "high", instructions[current_command]["last"].values()))):
                    for i in instructions[current_command]["value"]:
                        future_commands.append((i, current_command, "low"))
                else:
                    for i in instructions[current_command]["value"]:
                        future_commands.append((i, current_command, "high"))

            elif instructions[current_command]["type"] == "%": #flipflop
                if signal == "low":
                    if instructions[current_command]["state"] == "off":
                        instructions[current_command]["state"] = "on"
                        for i in instructions[current_command]["value"]:
                            future_commands.append((i, current_command, "high"))
                    else:
                        instructions[current_command]["state"] = "off"
                        for i in instructions[current_command]["value"]:
                            future_commands.append((i, current_command, "low"))
        #print(future_commands)
        if len(future_commands) > 0:
            next_command = future_commands.pop(0)
            current_command = next_command[0]
            source = next_command[1]
            signal = next_command[2]
        else:
            running = False

    return False

instructions = {}
for line in lines:
    instruction = line.strip().split(' -> ')
    if instruction[0] == "broadcaster":
        instructions[instruction[0]] = {"type": "broadcaster", "value": instruction[1].split(", ")}
    elif instruction[0][0] == "%":
        instructions[instruction[0][1:]] = {"type": instruction[0][0], "value": instruction[1].split(", "), "state": "off"}#flipflop
    elif instruction[0][0] == "&":
        instructions[instruction[0][1:]] = {"type": instruction[0][0], "value": instruction[1].split(", "), "last": {}}#Conjunction

for key in instructions:
    if instructions[key]["type"] == "&":
        for i in list(dict(filter(lambda x: key in x[1]["value"], instructions.items())).keys()):
            instructions[key]["last"][i] = "low"

counters = []
for i in instructions["broadcaster"]["value"]:
    cnt = 0
    found = False
    while not found:
        found = process_button(i, "broadcaster", "low", [])
        cnt += 1
    counters.append(cnt)
print(f"Part 2: {reduce(lambda x, y: lcm(x, y), counters)}")


Part 2: 231657829136023
